## Fill in Automatically

Fill in the missing columns of the data frame automatically. This is mostly information that is available online, or otherwise tedious to get.

In [2]:
import os

import pandas as pd
import numpy as np
import openpyxl

In [3]:
#Static
df_name = 'data_set_master_thesis_cala.xlsm'
lit_file_name = r'C:\Users\hso20\OneDrive\Plocha\IES\Diploma-Thesis\Literature\Literature.xlsx'
if not os.path.exists(df_name):
    raise ValueError('Missing data frame')

# Read the lit file 
data_file = pd.read_excel(df_name, sheet_name = 'data_set')
country_file = pd.read_excel(df_name, sheet_name = 'country_info') # Information about country categorization
lit_file = pd.read_excel(lit_file_name, sheet_name='Query literature')


In [69]:
def getColumnIdx(active_sheet, col_name:str):
    '''Input an excel sheet, name of a column to look for in an excel spreadsheet
    and return its index as an integer. Break if it is not found.
    Args:
        active_sheet (sheet): Sheet to search in.
        col_name (str): Name of the column to look for.
    Returns:
        idx (int): Index of the column in the worksheet.
    '''
    idx_ = None
    # Iterate through all cells of the first row
    for idx, cell in enumerate(active_sheet[1]):
        if cell.value == col_name:
            idx_ = idx
        if idx_ is not None:
            break
    if idx is None:
        raise ValueError(f'{col_name} not found in {active_sheet}.')
    return idx_


def addCitations(active_sheet, lit_file:pd.DataFrame):
    '''Input the active excel_sheet, and the raw literature pandas data frame and return the desired array of citations.
    '''
    # Get the dictionary of citations for all studies
    rel_studies = lit_file[lit_file["Collected"] == "YES"]
    citations_dict = dict() # A list of dictionaries
    for study_iter in rel_studies.iterrows():
        study = study_iter[1]
        study_name = study["Label"]
        citations = study["Cited by"]
        citations_dict[study_name] = citations
    
    # Inside the active sheet, find the column indices for "study_name" and "citations"
    study_name_idx = getColumnIdx(active_sheet, 'study_name')
    citations_idx = getColumnIdx(active_sheet, 'citations')

    # Fill in the citations
    for row in active_sheet.iter_rows(min_row=2, values_only = True):
        study_name = row[study_name_idx]
        if study_name in citations_dict:
            cell = active_sheet.cell(row = row[0] + 1, column = citations_idx + 1) # Adding 1 because .cell() uses 1-indexing
            cell.value = citations_dict[study_name]
    
    return None

def addCountryInfo(active_sheet, countries_data:pd.DataFrame):
    '''Specify the active sheet of the workbook, the country information and fill in the categories:
    -Region type
    -Income type
    Args:
        active_sheet (sheet): Active sheet of the workbook.
        countries_data (pd.DataFrame): Information about the countries
    Note:
        Automatically validates that the countries are in the correct format.
    '''
    country_col_idx = getColumnIdx(active_sheet, 'country')
    region_advanced_economies_idx = getColumnIdx(active_sheet, 'region_advanced_economies')
    region_east_asia_and_pacific_idx = getColumnIdx(active_sheet, 'region_east_asia_and_pacific')
    region_europe_and_central_asia_idx = getColumnIdx(active_sheet, 'region_europe_and_central_asia')
    region_latin_america_and_caribbean_idx = getColumnIdx(active_sheet, 'region_latin_america_and_caribbean')
    region_middle_east_and_north_africa_idx = getColumnIdx(active_sheet, 'region_middle_east_and_north_africa')
    region_south_asia_idx = getColumnIdx(active_sheet, 'region_south_asia')
    region_sub_saharan_africa_idx = getColumnIdx(active_sheet, 'region_sub_saharan_africa')
    income_high_idx = getColumnIdx(active_sheet, 'income_high')
    income_middle_idx = getColumnIdx(active_sheet, 'income_middle')
    income_low_idx = getColumnIdx(active_sheet, 'income_low')
    source_country_names = pd.Series(countries_data.country)
    for row in active_sheet.iter_rows(min_row=2, values_only = True):
        country_name = row[country_col_idx]
        if country_name is not None: # Only until the end of the data frame
            # Check that country data is available
            if country_name not in source_country_names.values:
                raise ValueError(f'{country_name} is missing from the source data. Make sure to add it.')
            # Get all values for this row
            region_advanced_economies_value = int(countries_data.region_advanced_economies[countries_data["country"]==country_name])
            region_east_asia_and_pacific_value = int(countries_data.region_east_asia_and_pacific[countries_data["country"]==country_name])
            region_europe_and_central_asia_value = int(countries_data.region_europe_and_central_asia[countries_data["country"]==country_name])
            region_latin_america_and_caribbean_value = int(countries_data.region_latin_america_and_caribbean[countries_data["country"]==country_name])
            region_middle_east_and_north_africa_value = int(countries_data.region_middle_east_and_north_africa[countries_data["country"]==country_name])
            region_south_asia_value = int(countries_data.region_south_asia[countries_data["country"]==country_name])
            region_sub_saharan_africa_value = int(countries_data.region_sub_saharan_africa[countries_data["country"]==country_name])
            income_high_value = int(countries_data.income_high[countries_data["country"]==country_name])
            income_middle_value = int(countries_data.income_middle[countries_data["country"]==country_name])
            income_low_value = int(countries_data.income_low[countries_data["country"]==country_name])
            # Put those values into the new data frame
            def fillInCell(idx, val):
                if idx is None or val is None:
                    raise ValueError(f"Idx or Value is None in country {country_name}.")
                cell = active_sheet.cell(row = row[0] + 1, column = idx + 1)
                cell.value = val
            fillInCell(region_advanced_economies_idx, region_advanced_economies_value)
            fillInCell(region_east_asia_and_pacific_idx, region_east_asia_and_pacific_value)
            fillInCell(region_europe_and_central_asia_idx, region_europe_and_central_asia_value)
            fillInCell(region_latin_america_and_caribbean_idx, region_latin_america_and_caribbean_value)
            fillInCell(region_middle_east_and_north_africa_idx, region_middle_east_and_north_africa_value)
            fillInCell(region_south_asia_idx, region_south_asia_value)
            fillInCell(region_sub_saharan_africa_idx, region_sub_saharan_africa_value)
            fillInCell(income_high_idx, income_high_value)
            fillInCell(income_middle_idx, income_middle_value)
            fillInCell(income_low_idx, income_low_value)
    return None

# Possibly consider adding this into the country_info data frame, so that all country information can be traced into one place
def addEthnicityInfo(active_sheet):
    'Fill in ethnicity - ethiicity_caucasian', 'ethnicity_other'
    # A list of caucasian countries (generated by AI)
    causasian_countries = ['Albania', 'Andorra', 'Armenia', 'Austria', 'Azerbaijan', 'Belarus', 'Belgium', 'Bosnia and Herzegovina', 'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Georgia', 'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy', 'Kazakhstan', 'Kosovo', 'Latvia', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Macedonia (FYROM)', 'Malta', 'Moldova', 'Monaco', 'Montenegro', 'Netherlands', 'Norway', 'Poland', 'Portugal', 'Romania', 'Russia', 'San Marino', 'Serbia', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'Ukraine', 'United Kingdom', 'Vatican City']
    # Get col indexes
    country_col_idx = getColumnIdx(active_sheet, 'country')
    ethnicity_caucasian_idx = getColumnIdx(active_sheet, 'ethnicity_caucasian')
    ethnicity_other_idx = getColumnIdx(active_sheet, 'ethnicity_other')
    # Fill in 
    for row in active_sheet.iter_rows(min_row=2, values_only = True):
        country_name = row[country_col_idx]
        if country_name is not None: # Only until the end of the data frame
            new_value = 1 if country_name in causasian_countries else 0
            cell = active_sheet.cell(row = row[0] + 1, column = ethnicity_caucasian_idx + 1)
            cell.value = new_value # Is caucasian
            cell = active_sheet.cell(row = row[0] + 1, column = ethnicity_other_idx + 1)
            cell.value = 1 - new_value # Is not caucasian
    return None





In [70]:
# Load the workbook and select the sheet
wb = openpyxl.load_workbook(df_name)
study_sheet = wb['data_set'] # Data_set worksheet

# Add the citations
addCitations(study_sheet, lit_file)
addCountryInfo(study_sheet, country_file)
addEthnicityInfo(study_sheet)

wb.save('updated_' + 'test_file.xlsx')